# Подготовка данных и обучение

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np

### Использовать буду только самые необходимые признаки.
1. Оценка разговора + флаг использования продукта из таблицы communications
2. Прохождение курсов пользователями из таблицы courses_passing

In [3]:
# Загружаем два датасета
communications = pd.read_csv(
    "../data/src/communications.csv", \
        sep=';', dtype={'employee_id': 'category'}, parse_dates=['communication_dt'])
courses_passing = pd.read_csv(
    "../data/src/courses_passing.csv", \
        sep=';', dtype={'employee_id': 'category'}, parse_dates=['start_dt', 'end_dt'])

### Идея заключается в следующем.
1. Отберём тех пользователей, которые курс окончили.
2. Зададим промежуток времени до курса и после курса.
3. Создадим взвешенную меру эффективности курса.
4. Разделим датасет по мере эффективности каждого курса <br>для каждого человека на "до" и "после"
5. Посчитаем разницу
6. Составим матрицу оценок
7. Обучим SVD

In [4]:
# Предлагаю считать только тех, кто курсы окончил
courses_passed = courses_passing[~courses_passing['end_dt'].isnull()]
cols_to_include = ['start_dt', 'course_id', 'employee_id', 'end_dt']
courses_passed = courses_passed.loc[:, cols_to_include]

In [5]:
# Создаём промежуток размером в месяц до начала курса
courses_passed['month_before'] = courses_passed['start_dt'] \
    - pd.DateOffset(days=30)
# Создаём промежуток размером в месяц после окончания курса
courses_passed['month_after'] = courses_passed['end_dt'] \
    + pd.DateOffset(days=30)

In [6]:
# Удаляем неинформативный атрибут
communications = communications.drop('communication_id', axis=1)

In [7]:
# Предлагаю считать оценку разговора по следующей формуле
communications['weighted_mark'] = \
    communications['util_flg'] * 100 * 0.5 \
    + communications['communication_score'] * 0.5
# Так мы учтём влияние прихода клиента в общей оценке,
# как немаловажный показатель эффективности труда.

In [8]:
# Объединяем две таблицы. Получаем для каждой оценки интервалы
# которые сможем использовать как "ДО" и "ПОСЛЕ"
merged = communications \
    .merge(courses_passed, how='inner', on='employee_id')

In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19129660 entries, 0 to 19129659
Data columns (total 10 columns):
 #   Column               Dtype         
---  ------               -----         
 0   communication_dt     datetime64[ns]
 1   employee_id          object        
 2   communication_score  int64         
 3   util_flg             int64         
 4   weighted_mark        float64       
 5   start_dt             datetime64[ns]
 6   course_id            int64         
 7   end_dt               datetime64[ns]
 8   month_before         datetime64[ns]
 9   month_after          datetime64[ns]
dtypes: datetime64[ns](5), float64(1), int64(3), object(1)
memory usage: 1.4+ GB


In [10]:
# Удалим ненужное
del communications, courses_passed

In [11]:
# Удалим более ненужные признаки
merged = merged \
    .drop(['communication_score', 'util_flg'], axis=1)
# Просмотр первых нескольких строк объединенной таблицы
merged.head()

,communication_dt,employee_id,weighted_mark,start_dt,course_id,end_dt,month_before,month_after
0,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-06-25,79,2023-07-05,2023-05-26,2023-08-04
1,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-07-06,74,2023-07-15,2023-06-06,2023-08-14
2,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-06-03,16,2023-06-09,2023-05-04,2023-07-09
3,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-07-13,8,2023-07-20,2023-06-13,2023-08-19
4,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-04-09,42,2023-04-15,2023-03-10,2023-05-15


In [12]:
# Коммуникации, которые были после курса и в пределах месяца
# следующего как контрольный после конца курса.
is_after = (merged['communication_dt'] > merged['end_dt'])\
    & (merged['communication_dt'] < merged['month_after'])
    
# Коммуникации, которые были до курса и в пределах месяца
# следующего как контрольный до начала курса.
is_before = (merged['communication_dt'] < merged['start_dt']) \
    & (merged['communication_dt'] > merged['month_before'])

In [13]:
# Считаем среднюю оценку до курса и после
mark_after_course = merged[is_after] \
    .groupby(['employee_id', 'course_id'])['weighted_mark'] \
        .agg('mean')
mark_before_course = merged[is_before] \
    .groupby(['employee_id', 'course_id'])['weighted_mark'] \
        .agg('mean')

In [14]:
mark_before_course

employee_id                           course_id
00411460-f7c9-2d21-24a6-7ea0f4cb5f85  7            59.918919
                                      8            67.408088
                                      20           67.141256
                                      29           68.684896
                                      50           65.909091
                                                     ...    
ffeed84c-7cb1-ae7b-f4ec-4bd78275bb98  46           70.426966
                                      51           66.660000
                                      74           67.276190
                                      79           70.024457
                                      82           66.654321
Name: weighted_mark, Length: 8404, dtype: float64

In [15]:
# Считаем разницу
diff_df = (mark_after_course - mark_before_course)
diff_df = diff_df.unstack()
# Смотрим что получилось
diff_df.head()

course_id,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
employee_id,,,,,,,,,,,,,,,,,,,,,
00411460-f7c9-2d21-24a6-7ea0f4cb5f85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.593618,-0.60589,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006f52e9-102a-8d3b-e2fe-5614f42ba989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.588911,NaN,NaN,NaN,NaN
00a03ec6-533c-a7f5-c644-d198d815329c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-5.523111,NaN,NaN,NaN,NaN,NaN,7.075201,NaN,NaN
00ac8ed3-b432-7bdd-4ebb-ebcb2ba10a00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00e26af6-ac3b-1c1c-49d7-c3d79c60d000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.128442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
diff_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1677 entries, 00411460-f7c9-2d21-24a6-7ea0f4cb5f85 to ffeed84c-7cb1-ae7b-f4ec-4bd78275bb98
Data columns (total 92 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       30 non-null     float64
 1   1       211 non-null    float64
 2   2       30 non-null     float64
 3   3       26 non-null     float64
 4   4       54 non-null     float64
 5   5       55 non-null     float64
 6   6       57 non-null     float64
 7   7       130 non-null    float64
 8   8       288 non-null    float64
 9   9       83 non-null     float64
 10  10      215 non-null    float64
 11  11      53 non-null     float64
 12  12      38 non-null     float64
 13  13      25 non-null     float64
 14  14      110 non-null    float64
 15  15      51 non-null     float64
 16  16      222 non-null    float64
 17  17      43 non-null     float64
 18  18      105 non-null    float64
 19  19      49 non-null     float64
 20  20      307 

### Небольшой subtotal.
1. Мы вычислили средние взвешенные оценки разговоров пользователей.
2. Посчитали разницу во взвешенных оценках в промежутке "до" и "после"

Теперь будем приводить к единому масштабу.

In [17]:
def scale_row(row):
    min_val = row.min()
    max_val = row.max()
    if pd.isna(min_val) or pd.isna(max_val) or min_val == max_val:
        return row
    else:
        return (row - min_val) / (max_val - min_val) * 9 + 1


# Применяем функцию к каждой строке
diff_df = diff_df.apply(scale_row, axis=1)

diff_df = diff_df.fillna(0)

In [18]:
diff_df.iloc[0, :].min()

0.0

In [19]:
# удалим лишнее из памяти
del merged

In [20]:
# Выполнение SVD разложения
U, sigma, Vt = np.linalg.svd(diff_df, full_matrices=False)

# Восстановление оценок
reconstructed_data = np.dot(U[:, :len(sigma)] * sigma, Vt)
reconstructed_data

array([[-4.66947832e-16, -1.95990653e-15, -1.03786898e-15, ...,
        -1.27295409e-15,  1.69920739e-16, -1.34267257e-15],
       [ 8.64377741e-16,  1.32663265e-14, -4.24755163e-15, ...,
        -4.86622032e-15, -4.70144964e-15, -1.92295307e-15],
       [ 1.14892803e-15, -3.27803310e-15,  1.62152455e-15, ...,
         7.36230940e+00,  6.68404557e-16,  1.54188497e-15],
       ...,
       [ 3.35209126e-15,  3.39798758e-15, -7.02573684e-16, ...,
         6.58906988e-15,  4.95757634e-15,  5.26438485e-15],
       [ 7.81801064e-16,  9.47092662e-16, -3.66513919e-17, ...,
         4.69463394e+00,  1.63515110e-15, -2.31174984e-16],
       [ 3.52587832e-16, -1.15944418e-15,  1.10908720e-19, ...,
        -2.13683862e-15,  1.97969667e-15, -1.20926263e-15]])

In [21]:
svd_matrix = pd.DataFrame(data=reconstructed_data,
             index=diff_df.index, columns=diff_df.columns)

In [22]:
svd_matrix

course_id,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
employee_id,,,,,,,,,,,,,,,,,,,,,
00411460-f7c9-2d21-24a6-7ea0f4cb5f85,-4.669478e-16,-1.959907e-15,-1.037869e-15,2.438121e-15,4.041164e-16,-2.216496e-15,1.424688e-15,4.165673e+00,4.151556e+00,2.599187e-15,...,4.377786e-15,1.805481e-15,1.518681e-15,-2.729949e-15,-3.542847e-16,-1.187674e-15,8.729003e-16,-1.272954e-15,1.699207e-16,-1.342673e-15
006f52e9-102a-8d3b-e2fe-5614f42ba989,8.643777e-16,1.326633e-14,-4.247552e-15,-3.623865e-15,3.296053e-15,1.022998e-16,3.436277e-15,7.705179e-15,3.120732e-15,-5.188209e-17,...,1.985611e-15,4.761899e-15,6.503495e-16,8.408879e-16,5.134177e-15,5.118893e+00,2.185100e-15,-4.866220e-15,-4.701450e-15,-1.922953e-15
00a03ec6-533c-a7f5-c644-d198d815329c,1.148928e-15,-3.278033e-15,1.621525e-15,-8.938564e-15,2.845948e-15,6.061717e-15,-2.942057e-15,-9.963992e-15,7.257150e-15,6.515754e-15,...,6.959371e-15,1.000000e+00,-2.477471e-15,6.068510e-15,-1.432827e-15,9.995917e-16,9.110963e-15,7.362309e+00,6.684046e-16,1.541885e-15
00ac8ed3-b432-7bdd-4ebb-ebcb2ba10a00,9.042283e-16,-5.821937e-16,-1.357531e-15,1.633248e-15,-2.631805e-15,5.733759e-16,-1.482328e-15,-4.080599e-15,7.127368e-15,3.327697e-15,...,-3.051884e-16,-7.977767e-16,3.444950e-16,-3.119382e-16,-6.326584e-16,2.049184e-15,-5.165555e-16,3.753988e-16,1.439481e-14,-1.750458e-15
00e26af6-ac3b-1c1c-49d7-c3d79c60d000,2.300149e-15,-1.348608e-15,1.271581e-15,4.203151e-15,4.340592e-17,-1.446777e-15,2.717552e-15,3.440366e-15,8.841481e-15,6.060751e-16,...,-7.040285e-15,9.063501e+00,-2.001125e-15,9.972263e-16,3.603906e-16,3.636088e-15,-2.736592e-15,1.214801e-14,-1.318426e-14,-5.381721e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff1418e8-cc99-3fe8-abcf-e3ce2003e5c5,1.556602e-15,-5.417465e-16,1.981752e-15,5.822915e-16,3.239920e-15,1.786873e-16,1.390439e-15,1.488036e-15,7.887958e+00,2.083111e-15,...,1.530701e-15,1.235311e-14,-1.230187e-15,7.510280e-16,2.692280e-15,6.070589e-16,6.275123e-16,1.000000e+00,-9.255799e-16,2.004887e-15
ff49cc40-a889-0e6a-60f4-0ff3026d2730,2.583085e-16,-1.286959e-15,6.297426e-16,5.457667e-16,-7.370169e-16,-7.609514e-16,-9.698331e-17,1.372817e-15,-2.728621e-15,-4.039544e-16,...,-1.443929e-15,-1.173149e-15,-8.073449e-16,4.511888e-16,4.143504e-16,-4.213437e-16,8.392137e-16,4.928568e-16,-9.091888e-17,3.576649e-16
fface838-5abb-f94b-4593-a0ed53a0c70f,3.352091e-15,3.397988e-15,-7.025737e-16,-3.808100e-15,1.795074e-15,7.479632e-17,1.370357e-15,1.244083e-15,1.081613e-14,1.964433e-15,...,8.670827e-15,-5.240375e-16,8.332251e-16,-3.800981e-15,3.076138e-15,6.744931e-15,2.122657e-15,6.589070e-15,4.957576e-15,5.264385e-15


In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Вычисление среднеквадратичной ошибки
mse = mean_squared_error(diff_df, reconstructed_data)
print("Mean Squared Error (MSE):", mse)

# Вычисление средней абсолютной ошибки
mae = mean_absolute_error(diff_df, reconstructed_data)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 2.432825886226364e-29
Mean Absolute Error (MAE): 2.4546757876869398e-15


#### Обе метрики близки к нулю, поэтому модель довольно хорошо реконструировала исходные данные, а значит может быть использована в качестве рекомендаций

In [19]:
# Рекомендации топ рекомендаций для первого пользователя
svd_matrix.iloc[0, :].argsort().sort_values(ascending=False)

course_id
75    91
69    90
22    89
9     88
24    87
      ..
71     4
53     3
72     2
37     1
28     0
Name: 00411460-f7c9-2d21-24a6-7ea0f4cb5f85, Length: 92, dtype: int64

### Интерпретация
Для пользователя под id "00411460-f7c9-2d21-24a6-7ea0f4cb5f85" наиболее рекомендуемый курс под id 75, на втором месте под id 69 и так далее